In [1]:
from medgan_model import Medgan
from data_loader import load_nursery_data
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from medgan_model import Medgan
from data_loader import load_shuttle_data
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import pandas as pd
from sklearn.metrics import classification_report

In [2]:
import pandas as pd
df = pd.read_csv("datasets/nursery.csv")
print(df.columns.tolist())


['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'Target']


In [8]:
# --- Load Nursery Dataset ---
x_train, x_test, y_train, y_test = load_nursery_data("datasets/nursery.csv")
input_dim = x_train.shape[1]

# --- Initialize MedGAN ---
medgan = Medgan(input_dim=input_dim, ae_loss_type='bce')

# --- Initialize and Train MedGAN ---
medgan = Medgan(input_dim=input_dim, ae_loss_type='bce')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
batch_size = 128
n_epochs = 300

In [9]:
for epoch in range(n_epochs):
    np.random.shuffle(x_train)
    for i in range(0, len(x_train), batch_size):
        batch = x_train[i:i + batch_size]
        noise = np.random.normal(size=(batch.shape[0], medgan.random_dim))
        medgan.train_step(batch, noise)
    print(f"Epoch {epoch+1}/{n_epochs} completed")

Epoch 1/300 completed
Epoch 2/300 completed
Epoch 3/300 completed
Epoch 4/300 completed
Epoch 5/300 completed
Epoch 6/300 completed
Epoch 7/300 completed
Epoch 8/300 completed
Epoch 9/300 completed
Epoch 10/300 completed
Epoch 11/300 completed
Epoch 12/300 completed
Epoch 13/300 completed
Epoch 14/300 completed
Epoch 15/300 completed
Epoch 16/300 completed
Epoch 17/300 completed
Epoch 18/300 completed
Epoch 19/300 completed
Epoch 20/300 completed
Epoch 21/300 completed
Epoch 22/300 completed
Epoch 23/300 completed
Epoch 24/300 completed
Epoch 25/300 completed
Epoch 26/300 completed
Epoch 27/300 completed
Epoch 28/300 completed
Epoch 29/300 completed
Epoch 30/300 completed
Epoch 31/300 completed
Epoch 32/300 completed
Epoch 33/300 completed
Epoch 34/300 completed
Epoch 35/300 completed
Epoch 36/300 completed
Epoch 37/300 completed
Epoch 38/300 completed
Epoch 39/300 completed
Epoch 40/300 completed
Epoch 41/300 completed
Epoch 42/300 completed
Epoch 43/300 completed
Epoch 44/300 complet

In [10]:
# --- Generate Synthetic Data ---
synthetic_data = medgan.generate_data(num_samples=1000)

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# --- Encode labels ---
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(pd.concat([y_train, y_test], axis=0))

y_train_real = label_encoder.transform(y_train)
y_test_real = label_encoder.transform(y_test)

# --- Classifier Definitions ---
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "MLP Classifier": MLPClassifier(max_iter=300, random_state=42),
    "KNN Classifier": Pipeline([
        ('scaler', StandardScaler()),  # Scaling needed for KNN
        ('knn', KNeighborsClassifier(n_neighbors=5))
    ]),
    "Logistic Regression": LogisticRegression(max_iter=300, random_state=42)
}

# --- Evaluation ---
results = {}

for name, model in models.items():
    model.fit(x_train, y_train_real)
    y_pred = model.predict(x_test)

    results[name] = {
        "Accuracy": accuracy_score(y_test_real, y_pred),
        "Precision": precision_score(y_test_real, y_pred, average='macro', zero_division=0),
        "Recall": recall_score(y_test_real, y_pred, average='macro', zero_division=0),
        "F1 Score": f1_score(y_test_real, y_pred, average='macro', zero_division=0)
    }

results_df = pd.DataFrame(results).T
print("\nClassifier Evaluation on Real Data:")
print(results_df)


C:\Anaconda\envs\medgan_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(



Classifier Evaluation on Real Data:
                     Accuracy  Precision    Recall  F1 Score
Random Forest        0.328318   0.196176  0.201306  0.198583
MLP Classifier       0.325231   0.194906  0.199203  0.196684
KNN Classifier       0.339120   0.199211  0.205586  0.198150
Logistic Regression  0.325617   0.180787  0.195963  0.182792
